# Sampling

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import dependencies
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from collections import Counter

In [3]:
# Load the data
file_path = Path("resources/processed_stroke_db_w_cat.csv")
df = pd.read_csv(file_path, index_col=0)
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
14180,Female,13.0,0,0,No,children,Rural,103.08,18.6,Unknown,0
44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0


### One-hot Encoding for Categotical Data & StandScaler for Numerical Data

In [4]:
# Generate our categorical variable list
cat_labels= df.dtypes[df.dtypes == "object"].index.tolist()
cat_labels

['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

In [5]:
# Check the number of unique values in each column
df[cat_labels].nunique()

gender            3
ever_married      2
work_type         5
Residence_type    2
smoking_status    4
dtype: int64

In [6]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [8]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat_labels]))

In [9]:
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat_labels)
encode_df.head()

,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(cat_labels,1)
df.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
1665,79.0,1,0,174.12,24.0,1,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4219,71.0,0,0,102.87,27.2,1,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
712,82.0,1,1,84.03,26.5,1,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4651,78.0,0,0,78.03,23.9,1,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1261,54.0,0,0,71.22,28.5,1,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = df["stroke"].values
X = df.drop(["stroke"],1).values

In [12]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [13]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [14]:
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

In [15]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Oversampling

### Naive Random

In [16]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
# Instantiate the model
ros = RandomOverSampler(random_state=1)
# Resample the targets
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 208, 1: 208})

In [17]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs', random_state=1)

# Fit
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
from sklearn.metrics import balanced_accuracy_score
bas = balanced_accuracy_score(y_test, y_pred)
bas

0.9202898550724637

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Stroke", "Actual No Stroke"], columns=["Predicted Stroke", "Predicted No Stroke"])
cm_df

,Predicted Stroke,Predicted No Stroke
Actual Stroke,58,11
Actual No Stroke,0,5


In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.84      1.00      0.91      0.92      0.83        69
          1       0.31      1.00      0.84      0.48      0.92      0.85         5

avg / total       0.95      0.85      0.99      0.88      0.92      0.83        74



In [21]:
print("Naive Random Oversampling")
print(f"Balanced Accuracy Score: {bas}")
print(cm_df)
print(classification_report_imbalanced(y_test, y_pred))

Naive Random Oversampling
Balanced Accuracy Score: 0.9202898550724637
                  Predicted Stroke  Predicted No Stroke
Actual Stroke                   58                   11
Actual No Stroke                 0                    5
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.84      1.00      0.91      0.92      0.83        69
          1       0.31      1.00      0.84      0.48      0.92      0.85         5

avg / total       0.95      0.85      0.99      0.88      0.92      0.83        74



### SMOTE

In [22]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
# Instantiate the model
smote = SMOTE(random_state=1)
# Resample the targets
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 208, 1: 208})

In [23]:
# Train the Logistic Regression model using the resampled data
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
bas = balanced_accuracy_score(y_test, y_pred)
bas

0.8202898550724638

In [25]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Stroke", "Actual No Stroke"], columns=["Predicted Stroke", "Predicted No Stroke"])
cm_df

,Predicted Stroke,Predicted No Stroke
Actual Stroke,58,11
Actual No Stroke,1,4


In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.98      0.84      0.80      0.91      0.82      0.68        69
          1       0.27      0.80      0.84      0.40      0.82      0.67         5

avg / total       0.93      0.84      0.80      0.87      0.82      0.67        74



In [27]:
print("SMOTE Oversampling")
print(f"Balanced Accuracy Score: {bas}")
print(cm_df)
print(classification_report_imbalanced(y_test, y_pred))

SMOTE Oversampling
Balanced Accuracy Score: 0.8202898550724638
                  Predicted Stroke  Predicted No Stroke
Actual Stroke                   58                   11
Actual No Stroke                 1                    4
                   pre       rec       spe        f1       geo       iba       sup

          0       0.98      0.84      0.80      0.91      0.82      0.68        69
          1       0.27      0.80      0.84      0.40      0.82      0.67         5

avg / total       0.93      0.84      0.80      0.87      0.82      0.67        74



## Undersampling

### Cluster Centroids

In [28]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
# Instantiate
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 13, 1: 13})

In [29]:
# Train the Logistic Regression model using the resampled data
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [30]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
bas = balanced_accuracy_score(y_test, y_pred)
bas

0.7275362318840579

In [31]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Stroke", "Actual No Stroke"], columns=["Predicted Stroke", "Predicted No Stroke"])
cm_df

,Predicted Stroke,Predicted No Stroke
Actual Stroke,59,10
Actual No Stroke,2,3


In [32]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.86      0.60      0.91      0.72      0.53        69
          1       0.23      0.60      0.86      0.33      0.72      0.50         5

avg / total       0.92      0.84      0.62      0.87      0.72      0.52        74



In [33]:
print("Cluster Centroids Undersampling")
print(f"Balanced Accuracy Score: {bas}")
print(cm_df)
print(classification_report_imbalanced(y_test, y_pred))

Cluster Centroids Undersampling
Balanced Accuracy Score: 0.7275362318840579
                  Predicted Stroke  Predicted No Stroke
Actual Stroke                   59                   10
Actual No Stroke                 2                    3
                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.86      0.60      0.91      0.72      0.53        69
          1       0.23      0.60      0.86      0.33      0.72      0.50         5

avg / total       0.92      0.84      0.62      0.87      0.72      0.52        74



## Combination (Over and Under) Sampling

### SMOTEEN

In [34]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smoteenn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 143, 1: 174})

In [35]:
# Train the Logistic Regression model using the resampled data
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [36]:
# Calculated the balanced accuracy score
y_pred = logreg.predict(X_test)
bas = balanced_accuracy_score(y_test, y_pred)
bas

0.8347826086956522

In [37]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Stroke", "Actual No Stroke"], columns=["Predicted Stroke", "Predicted No Stroke"])
cm_df

,Predicted Stroke,Predicted No Stroke
Actual Stroke,60,9
Actual No Stroke,1,4


In [38]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.98      0.87      0.80      0.92      0.83      0.70        69
          1       0.31      0.80      0.87      0.44      0.83      0.69         5

avg / total       0.94      0.86      0.80      0.89      0.83      0.70        74



In [39]:
print("SMOTEENN")
print(f"Balanced Accuracy Score: {bas}")
print(cm_df)
print(classification_report_imbalanced(y_test, y_pred))

SMOTEENN
Balanced Accuracy Score: 0.8347826086956522
                  Predicted Stroke  Predicted No Stroke
Actual Stroke                   60                    9
Actual No Stroke                 1                    4
                   pre       rec       spe        f1       geo       iba       sup

          0       0.98      0.87      0.80      0.92      0.83      0.70        69
          1       0.31      0.80      0.87      0.44      0.83      0.69         5

avg / total       0.94      0.86      0.80      0.89      0.83      0.70        74

